## 18.3 Homework

Try different minimization methods in scipy on larger systems ($N$ up to 20), and show 
- 1 the average number of attempts to find the ground state
- 2 the time costs
- 3 Write a function to compute the analytical gradient of total energy.


### Optional
try to improve the code to make it run faster, analyze the most time consuming part and give your solution

First, I define the functions LJ, total_energy, and init_pos that we used in lecture. Respectively, these calculate the LJ potential for atom pairs, the total energy for a configuration of N atoms, and a random initial position for atoms. To try to make computations quicker for configurations of N=4 to N=20 atoms, I import jit from numba.

In [21]:
import numpy as np
from numba import jit

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(positions):
    """
    Calculate the total energy
    input:
    positions: 3*N array which represents the atomic positions
    output
    E: the total energy
    """
    E = 0
    N_atom = int(len(positions)/3)

    #positions = [x0, y0, z0, x1, y1, z1, .....  , x(n-1), y(n-1), z(n-1)]
    for i in range(N_atom-1):
        for j in range(i+1, N_atom):
            pos1 = positions[i*3:(i+1)*3]
            pos2 = positions[j*3:(j+1)*3]
            #print('pos1:  ', pos1)
            #print('pos2:  ', pos2)
            dist = np.linalg.norm(pos1-pos2)
            #print(i,j, dist)
            E += LJ(dist)
    return E

@jit
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

Below is code very similar to the multi-step optimization code we covered in lecture. I create two lists which contain the number of atoms and true energies of each N-atom configuration, labelled "N_atoms" and "Energies" respectively. A for loops traverses through each N-atom configuration, and I output the number of steps taken until the while loop condition is met; in this case, I want to minimize the total energy of each configuration until I am at least within 0.8 of the known energy, and the number of "counts," or iterations is below or equal to 200. I chose a threshold energy difference of 0.8 to keep the number of counts as low as possible (better accuracy/tolerance will require more time and counts). For each N-atom configuration, I time how long the minimization takes and print this value. Since I am running the program for N_atoms from N=4 to N=20, I also have a global stopwatch to print the total time it takes to find the global minimum energy for all configurations in this range. Thus, in the output, we can see the number of steps it took to minimize each configuration, the time it took for computing the minimum, and the value of the global minimum for each configuration. Finally, we want to compare the different methods of minimization within scipy's minimize function; I repeat this whole process using three methods in this function: 'CG', 'BFGS', and 'L-BFGS-B'.

In [77]:
from scipy.optimize import minimize
import time


x_values = []
#N_attempts = 1
N_atoms = [4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
Energies = [-6.000000, -9.103852, -12.712062, -16.505384, -19.821489, -24.113360, -28.422532, -32.765970, 
-37.967600, -44.326801, -47.845157, -52.322627, -56.815742, -61.317995, -66.530949, 
-72.659782, -77.177043]
    

"""
minimize 

Method One: 'CG'

"""
globalstart = time.time()
for n in N_atoms:
    
    fun_values = []
    f_values = 0
    count = 0
    start = time.time()
    while (f_values-Energies[n-4]>0.8 and count < 200):
        pos = init_pos(n)
        res = minimize(total_energy, pos, method='CG', tol=1e-4)
        f_values = res.fun
        fun_values.append(res.fun)
        #x_values.append(res.x)
        count +=1
        print('\r Step: {:d} out of {:d}; value: {:.4f} Time {:.2f} s'.format(count, 200 , res.fun, time.time()-start), flush=True, end='')
   # if i%10==0:
       # print('Step: ', i, '  values:', res.fun)
    if (f_values-Energies[n-4]<= 0.8):
        print('\n The global minimum for N =  ', n,  f_values)
    else:
        print('\n The global minimum for N = ', n, 'is', min(fun_values))
    
print('Total time taken: ', time.time()-globalstart, 's')

 Step: 1 out of 200; value: -6.0000 Time 0.03 s
 The global minimum for N =   4 -5.999999999902069
 Step: 1 out of 200; value: -9.1039 Time 0.06 s
 The global minimum for N =   5 -9.103852415479553
 Step: 1 out of 200; value: -12.3029 Time 0.07 s
 The global minimum for N =   6 -12.30292752942133
 Step: 13 out of 200; value: -16.5054 Time 1.75 s
 The global minimum for N =   7 -16.505384167973062
 Step: 1 out of 200; value: -19.8215 Time 0.17 s
 The global minimum for N =   8 -19.821489192024302
 Step: 3 out of 200; value: -24.1134 Time 0.88 s
 The global minimum for N =   9 -24.11336043352681
 Step: 1 out of 200; value: -28.4225 Time 0.34 s
 The global minimum for N =   10 -28.42253189322234
 Step: 23 out of 200; value: -32.7660 Time 11.53 s
 The global minimum for N =   11 -32.7659700897635
 Step: 8 out of 200; value: -37.9676 Time 4.60 s
 The global minimum for N =   12 -37.96759956207197
 Step: 10 out of 200; value: -44.3268 Time 8.05 s
 The global minimum for N =   13 -44.32680141

In [74]:
globalstart = time.time()
for n in N_atoms:
    
    fun_values = []
    f_values = 0
    count = 0
    start = time.time()
    while (f_values-Energies[n-4]>0.8 and count < 200):
        pos = init_pos(n)
        res = minimize(total_energy, pos, method='BFGS', tol=1e-4)
        f_values = res.fun
        fun_values.append(res.fun)
        #x_values.append(res.x)
        count +=1
        print('\r Step: {:d} out of {:d}; value: {:.4f} Time {:.2f} s'.format(count, 200 , res.fun, time.time()-start), flush=True, end='')
   # if i%10==0:
       # print('Step: ', i, '  values:', res.fun)
    if (f_values-Energies[n-4]<= 0.8):
        print('\n The global minimum for N =  ', n,  f_values)
    else:
        print('\n The global minimum for N = ', n, 'is', min(fun_values))
    
print('Total time taken: ', time.time()-globalstart, 's')

 Step: 1 out of 200; value: -6.0000 Time 0.08 s
 The global minimum for N =   4 -5.99999999999914
 Step: 1 out of 200; value: -9.1039 Time 0.07 s
 The global minimum for N =   5 -9.103852415671215
 Step: 1 out of 200; value: -12.3029 Time 0.09 s
 The global minimum for N =   6 -12.302927529577593
 Step: 4 out of 200; value: -15.9350 Time 0.35 s
 The global minimum for N =   7 -15.93504306036322
 Step: 1 out of 200; value: -19.8215 Time 0.13 s
 The global minimum for N =   8 -19.821489192076456
 Step: 5 out of 200; value: -24.1134 Time 0.85 s
 The global minimum for N =   9 -24.113360433606495
 Step: 92 out of 200; value: -28.4225 Time 18.92 s
 The global minimum for N =   10 -28.42253189341413
 Step: 44 out of 200; value: -32.7660 Time 12.60 s
 The global minimum for N =   11 -32.765970089936786
 Step: 14 out of 200; value: -37.9676 Time 6.39 s
 The global minimum for N =   12 -37.96759956221145
 Step: 32 out of 200; value: -44.3268 Time 22.34 s
 The global minimum for N =   13 -44.326

In [75]:
globalstart = time.time()
for n in N_atoms:
    
    fun_values = []
    f_values = 0
    count = 0
    start = time.time()
    while (f_values-Energies[n-4]>0.8 and count < 200):
        pos = init_pos(n)
        res = minimize(total_energy, pos, method='L-BFGS-B', tol=1e-4)
        f_values = res.fun
        fun_values.append(res.fun)
        #x_values.append(res.x)
        count +=1
        print('\r Step: {:d} out of {:d}; value: {:.4f} Time {:.2f} s'.format(count, 200 , res.fun, time.time()-start), flush=True, end='')
   # if i%10==0:
       # print('Step: ', i, '  values:', res.fun)
    if (f_values-Energies[n-4]<= 0.8):
        print('\n The global minimum for N =  ', n,  f_values)
    else:
        print('\n The global minimum for N = ', n, 'is', min(fun_values))
    
print('Total time taken: ', time.time()-globalstart, 's')

 Step: 2 out of 200; value: -5.9999 Time 0.03 s
 The global minimum for N =   4 -5.999855971947463
 Step: 9 out of 200; value: -9.1038 Time 0.12 s
 The global minimum for N =   5 -9.103814729246182
 Step: 42 out of 200; value: -12.3026 Time 0.90 s
 The global minimum for N =   6 -12.302584638742502
 Step: 14 out of 200; value: -16.5050 Time 0.60 s
 The global minimum for N =   7 -16.504982542470678
 Step: 8 out of 200; value: -19.8209 Time 0.51 s
 The global minimum for N =   8 -19.820896961780818
 Step: 93 out of 200; value: -24.1129 Time 8.71 s
 The global minimum for N =   9 -24.112855770573617
 Step: 200 out of 200; value: -25.2846 Time 27.88 s
 The global minimum for N =  10 is -27.55548476763671
 Step: 74 out of 200; value: -32.7641 Time 13.91 s
 The global minimum for N =   11 -32.76411969004979
 Step: 135 out of 200; value: -37.9664 Time 34.81 s
 The global minimum for N =   12 -37.96640562731688
 Step: 16 out of 200; value: -44.3253 Time 5.26 s
 The global minimum for N =   13

The total time taken for the three methods are summarized below:

- CG: 2216 s
- BFGS: 1264 s
- L-BFGS-B: 586 s

Thus, for this application, the L-BFGS-B method in scipy's minimize package is the fastest of the three tested. For all three methods, configuratons that require the most iterations are (in general) those that begin to approach N=20 atoms.